<a href="https://colab.research.google.com/github/baggybawz21/Jobbies-Everywhere-/blob/main/Copy_of_Python_Route_Suggester_(Flask_%26_Folium).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import math
from datetime import datetime, timedelta
import random

import folium
import requests
from flask import Flask, render_template_string
from geopy.geocoders import Nominatim

# --- Configuration ---
# You can change the start address and target distance here
START_ADDRESS = "PH1 5FL, Perth, Scotland"
TARGET_DISTANCE_METERS = 3500  # 3.5km
# File to store the timestamp of the last generated route
STATE_FILE = "route_state.json"

# --- Flask App Initialization ---
app = Flask(__name__)
# A secret key is needed for session management, although not used directly here.
app.config['SECRET_KEY'] = 'a-very-secret-key'

# --- Core Logic Functions ---

def geocode_address(address):
    """
    Converts a physical address string into geographic coordinates (latitude, longitude).
    This is the Python equivalent of CoreLocation's CLGeocoder.
    Returns: A tuple (latitude, longitude) or None if not found.
    """
    try:
        geolocator = Nominatim(user_agent="route_suggester_app")
        location = geolocator.geocode(address)
        if location:
            print(f"Geocoded '{address}' to: ({location.latitude}, {location.longitude})")
            return (location.latitude, location.longitude)
        else:
            print(f"Could not geocode address: {address}")
            return None
    except Exception as e:
        print(f"An error occurred during geocoding: {e}")
        return None

def calculate_destination_coordinate(start_lat, start_lon, distance_meters, bearing_degrees):
    """
    Calculates a destination coordinate given a start point, distance, and bearing.
    This is a direct Python implementation of the logic in the Swift example.
    """
    earth_radius_meters = 6371000.0

    lat1 = math.radians(start_lat)
    lon1 = math.radians(start_lon)
    bearing = math.radians(bearing_degrees)

    lat2 = math.asin(math.sin(lat1) * math.cos(distance_meters / earth_radius_meters) +
                     math.cos(lat1) * math.sin(distance_meters / earth_radius_meters) * math.cos(bearing))

    lon2 = lon1 + math.atan2(math.sin(bearing) * math.sin(distance_meters / earth_radius_meters) * math.cos(lat1),
                             math.cos(distance_meters / earth_radius_meters) - math.sin(lat1) * math.sin(lat2))

    return (math.degrees(lat2), math.degrees(lon2))

def get_route_from_osrm(start_coords, end_coords, transport_mode="foot"):
    """
    Fetches route data from the public OSRM (Open Source Routing Machine) API.
    This is the equivalent of MapKit's MKDirections.
    Returns: A tuple (list of route coordinates, distance in meters) or (None, None).
    """
    # OSRM expects coordinates in lon,lat format
    start_lon, start_lat = start_coords[1], start_coords[0]
    end_lon, end_lat = end_coords[1], end_coords[0]

    url = f"http://router.project-osrm.org/route/v1/{transport_mode}/{start_lon},{start_lat};{end_lon},{end_lat}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if data.get('routes'):
            route = data['routes'][0]
            distance = route['distance']  # Distance in meters
            geometry = route['geometry']['coordinates']
            # OSRM geojson returns [lon, lat], folium needs [lat, lon], so we swap them
            route_coords = [(point[1], point[0]) for point in geometry]
            return route_coords, distance
        else:
            print(f"OSRM API did not return a route. Message: {data.get('message')}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error calling OSRM API: {e}")
        return None, None

def find_best_matching_route(start_coords):
    """
    Generates several candidate routes and picks the one closest to the target distance.
    """
    candidate_routes = []
    print(f"Searching for candidate routes near {start_coords}...")

    for i in range(10):  # Try 10 candidates for a good selection
        print(f"--- Candidate {i+1} ---")
        random_bearing = random.uniform(0, 360)
        # Straight line distance is shorter than route distance, so aim a bit lower
        random_distance = random.uniform(1500, 3200)

        dest_coords = calculate_destination_coordinate(start_coords[0], start_coords[1], random_distance, random_bearing)

        route_coords, distance = get_route_from_osrm(start_coords, dest_coords)

        if route_coords and distance:
            print(f"Found route with distance: {distance:.0f}m")
            candidate_routes.append({'coords': route_coords, 'distance': distance})

    if not candidate_routes:
        return None

    # Find the route with the smallest difference from the target distance
    best_route = min(candidate_routes, key=lambda r: abs(r['distance'] - TARGET_DISTANCE_METERS))
    print(f"Selected best route with distance: {best_route['distance']:.0f}m")
    return best_route

def read_state():
    """Reads the last route generation time from the state file."""
    if not os.path.exists(STATE_FILE):
        return None
    try:
        with open(STATE_FILE, 'r') as f:
            state = json.load(f)
            return datetime.fromisoformat(state['last_route_date'])
    except (json.JSONDecodeError, KeyError, FileNotFoundError):
        return None

def write_state():
    """Writes the current time as the last route generation time."""
    state = {'last_route_date': datetime.now().isoformat()}
    with open(STATE_FILE, 'w') as f:
        json.dump(state, f)

# --- Flask HTML Template ---
# For simplicity, the HTML is embedded in the Python script.
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Python Route Suggester</title>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { padding: 20px; background-color: #f8f9fa; }
        .container { max-width: 900px; margin: auto; background-color: #fff; padding: 30px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .map-container { height: 500px; margin-top: 20px; border-radius: 8px; }
        .alert { margin-top: 20px; }
    </style>
</head>
<body>
    <div class="container">
        <h1 class="text-center mb-4">Route Suggester</h1>
        <p class="text-center text-muted">A new route from {{ start_address }} of ~{{ target_km }}km every 3 days.</p>

        <div class="d-grid">
            <a href="/" class="btn btn-primary btn-lg">Suggest New Route</a>
        </div>

        {% if message %}
        <div class="alert alert-info text-center" role="alert">
            <strong>{{ message }}</strong>
        </div>
        {% endif %}

        {% if map_html %}
            <div class="map-container">
                {{ map_html|safe }}
            </div>
            <div class="alert alert-success mt-4">
                <h4 class="alert-heading">Route Details:</h4>
                <p><strong>Distance:</strong> {{ "%.2f"|format(distance/1000) }} km</p>
            </div>
        {% endif %}
    </div>
</body>
</html>
"""

# --- Flask Route ---

@app.route("/")
def index():
    """Main route for the web application."""
    message = None
    map_html = None
    route_details = None

    # Check the 3-day cooldown period
    last_route_date = read_state()
    if last_route_date and (datetime.now() - last_route_date) < timedelta(days=3):
        time_since = datetime.now() - last_route_date
        next_due_in = timedelta(days=3) - time_since
        days = next_due_in.days
        hours = next_due_in.seconds